In [1]:
import sys

sys.path.insert(0, "../data_gen/")
sys.path.insert(0, "../layer/")

import argparse
import os
import tensorflow as tf
from keras import backend as k
from hourglass import HourglassNet

Using TensorFlow backend.


In [4]:
batch_size = 8
model_path = '../../trained_models/'
resume = False
num_stacks = 2
epochs = 1
init_epoch = 0

In [6]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
    
# TensorFlow wizardry
config = tf.ConfigProto()
    
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 1.0
    
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

xnet = HourglassNet(num_classes=16, num_stacks=num_stacks, num_channels=256, inres=(256, 256),outres=(64, 64))

if resume:
    xnet.resume_train(batch_size=batch_size,
                      model_path=model_path,
                      init_epoch=init_epoch, epochs=args.epochs)
else:
    xnet.build_model(show=True)
    xnet.train(epochs=epochs, model_path=model_path, batch_size=batch_size)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
front_conv_7x7_x1 (Conv2D)      (None, 128, 128, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 64) 256         front_conv_7x7_x1[0][0]          
__________________________________________________________________________________________________
front_residual_x1convBlock_conv (None, 128, 128, 64) 4160        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_norm

Epoch 1/1
   5/2780 [..............................] - ETA: 26:34:22 - loss: 2.7111 - 0_head_conv_1x1_parts_loss: 1.3393 - 1_head_conv_1x1_parts_loss: 1.3718 - 0_head_conv_1x1_parts_acc: 0.0599 - 1_head_conv_1x1_parts_acc: 0.0643

KeyboardInterrupt: 

In [4]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--gpuID", default=0, type=int, help='gpu id')
    parser.add_argument("--mobile", default=False, help="use depthwise conv in hourglass'")
    parser.add_argument("--batch_size", default=8, type=int, help='batch size for training')
    parser.add_argument("--model_path", help='path to store trained model')
    parser.add_argument("--num_stack", default=2, type=int, help='num of stacks')
    parser.add_argument("--epochs", default=20, type=int, help="number of traning epochs")
    parser.add_argument("--resume", default=False, type=bool, help="resume training or not")
    parser.add_argument("--resume_model", help="start point to retrain")
    parser.add_argument("--resume_model_json", help="model json")
    parser.add_argument("--init_epoch", type=int, help="epoch to resume")
    parser.add_argument("--tiny", default=False, type=bool, help="tiny network for speed, inres=[192x128], channel=128")
    
    args = parser.parse_args()
    
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpuID)
    
    # TensorFlow wizardry
    config = tf.ConfigProto()
    
    # Don't pre-allocate memory; allocate as-needed
    config.gpu_options.allow_growth = True

    # Only allow a total of half the GPU memory to be allocated
    config.gpu_options.per_process_gpu_memory_fraction = 1.0
    
    # Create a session with the above options specified.
    k.tensorflow_backend.set_session(tf.Session(config=config))
    
    if args.tiny:
        pass
    else:
        xnet = HourglassNet(num_classes=16, num_stacks=args.num_stack, num_channels=256, inres=(256, 256),outres=(64, 64))
        
    if args.resume:
        xnet.resume_train(batch_size=args.batch_size, model_json=args.resume_model_json,
                          model_weights=args.resume_model,
                          init_epoch=args.init_epoch, epochs=args.epochs)
    else:
        xnet.build_model(mobile=args.mobile, show=True)
        xnet.train(epochs=args.epochs, model_path=args.model_path, batch_size=args.batch_size)

usage: ipykernel_launcher.py [-h] [--gpuID GPUID] [--mobile MOBILE]
                             [--batch_size BATCH_SIZE]
                             [--model_path MODEL_PATH] [--num_stack NUM_STACK]
                             [--epochs EPOCHS] [--resume RESUME]
                             [--resume_model RESUME_MODEL]
                             [--resume_model_json RESUME_MODEL_JSON]
                             [--init_epoch INIT_EPOCH] [--tiny TINY]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\cxhrz\AppData\Roaming\jupyter\runtime\kernel-1362baf6-1788-4d33-a548-c5ea1fd8df6b.json


SystemExit: 2